# **rinna版GPT-2のテキスト生成(colab)** 


# **Driveを1度importしてから，ランタイムの再起動の実行**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

In [ ]:
!git clone https://github.com/huggingface/transformers -b v4.4.2

In [ ]:
%cd '/content/drive/My Drive/work/'

In [ ]:
!pip install -e transformers
!pip install datasets
!pip install sentencepiece

# **run_clm.pyの変更**
 ./transformers/examples/language-modeling/run_clm.py<br>
## 追加
from transformers import T5Tokenizer<br>
## 変更
check_min_version("4.4.2")<br>
tokenizer = T5Tokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)<br>
tokenizer = T5Tokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)<br>


## **ファインチューニングの実行**
work内にtrain.txtを置く


In [ ]:
%%time

# 事前学習の実行
!python ./transformers/examples/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=train.txt \
    --validation_file=train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=3 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=output3/ \
    --use_fast_tokenizer=False

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM
import csv

#入力文字
enter = ['張り地の素材は本革なので', '張り地の素材は合成皮革なので', '張り地の素材は綿なので', '張り地の素材はファブリックなので', '張り地には本革を用いているので', '張り地には合成皮革を用いているので', '張り地には綿を用いているので', '張り地にはファブリックを用いているので']
name = 'hariji'
num = 1 

header = ["入力","出力"]
with open("{0}_gen.csv".format(name), 'w', encoding="sjis") as f:
  writer = csv.writer(f, lineterminator="\n")
  writer.writerow(header)

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("output/")

for sentence in enter:
  print(num)
  # 推論
  input = tokenizer.encode(sentence, return_tensors="pt")
  output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=25)
  outer = tokenizer.batch_decode(output)

  #</s>の削除
  outer = [s.replace('</s> ','') for s in outer]
  length = len(outer)
  # for i in range(length):
  #   output[i] = output[i].replace('</s> ','')
  #   print(output[i])

  #CSV書き込み
  header = ["入力","出力"]
  with open("{0}_gen.csv".format(name), 'a', encoding="sjis") as f:
    writer = csv.writer(f, lineterminator="\n")
    one_row = [sentence, '']
    writer.writerow(one_row)
    for i in range(length):
      one_row = ["", outer[i]]
      writer.writerow(one_row)
  num += 1

print('complete')

CSV書き込み作成

In [ ]:
import csv
header = ["入力","出力"]
with open("output.csv", 'w', encoding="sjis") as f:
  writer = csv.writer(f, lineterminator="\n")
  length = len(outer)
  for i in range(length):
    print(outer[i])
    
    one_row = ["", outer[i]]
    writer.writerow(one_row)

In [ ]:
!pip freeze